# PROJET IA for HumanForYou

|Auteur|
|---|
|G. DUBOYS DE LAVIGERIE|
|T. VILETTE|
|O. BOUSSARD|
|A. BRICON|

## Objectifs du Livrable

Le présent document constitue le premier livrable du projet IA pour HumanForYou, visant à analyser et traiter les différentes données fournies par l'entreprise dans le but de comprendre les facteurs influençant le taux de rotation des employés. Les objectifs principaux de ce livrable sont les suivants :

1. **Compréhension des données :** Explorer et comprendre les différentes sources de données fournies par HumanForYou, y compris les données des ressources humaines, les évaluations des managers, les enquêtes sur la qualité de vie au travail et les horaires de travail.

2. **Prétraitement des données :** Nettoyer les données en éliminant les valeurs manquantes, en identifiant et en traitant les éventuelles erreurs ou incohérences, et en préparant les données pour l'analyse et la modélisation.

3. **Exploration des données :** Effectuer une analyse exploratoire des données pour identifier les tendances, les relations et les motifs significatifs pouvant influencer le taux de rotation des employés.

4. **Visualisation des données :** Utiliser des techniques de visualisation de données pour représenter graphiquement les principales caractéristiques des données et faciliter la compréhension des résultats.

5. **Préparation des données pour la modélisation :** Préparer les données en sélectionnant les variables pertinentes, en transformant les variables catégorielles en variables numériques, et en divisant les données en ensembles d'entraînement et de test pour la modélisation.

## Attendus

À la fin de ce livrable, nous nous attendons à ce que les données fournies par HumanForYou soient prêtes à être analysées et utilisées pour la modélisation. Nous fournirons une analyse détaillée de la qualité des données, des tendances et des relations identifiées, ainsi qu'une documentation complète sur les étapes de prétraitement des données.

### 1. Préparation de l'environnement

Pour faciliter l'importation des bibliothèques nécessaires au bon fonctionnement du code, veuillez exécuter le fichier `setup.ipynb`. Ce fichier s'occupe de configurer l'environnement en important toutes les librairies essentielles. 

In [ ]:
# compatibilité python 2 et python 3
from __future__ import division, print_function, unicode_literals

# imports
import numpy as np
import os

# stabilité du notebook d'une exécution à l'autre
np.random.seed(42)

# jolies figures directement dans le notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# où sauver les figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "workflowDS"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID) # le dossier doit exister

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# ignorer les warnings inutiles (voir SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

### 2. Import des données

Dans cette section, nous automatisons le processus d'importation des données en implémentant une fonction qui effectue les étapes suivantes :
1. Téléchargement de l'archive contenant les fichiers.
2. Extraction des fichiers de l'archive.

Le code ci-dessous réalise le chargement des fichiers suivants :
- `employee_survey_data.csv`
- `general_data.csv`
- `in_time.csv`
- `out_time.csv`
- `manager_survey_data.csv`

De même, on va créer une fonction utilisant [`pandas`](https://pandas.pydata.org/) qui charge les données en mémoire dans un [`Pandas DataFrame`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#pandas.DataFrame).

In [ ]:
import os
import zipfile
from six.moves import urllib
import pandas as pd
from sklearn.impute import SimpleImputer

pd.set_option('display.max_columns', None)

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/"
REPO_PATH = "AnatholyB1/AI_A4/main/"
DATA_PATH = os.path.join("../datasets", "all")
DATA_URL = DOWNLOAD_ROOT + REPO_PATH + "data.zip"


def fetch_data(data_url=DATA_URL, data_path=DATA_PATH):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    zip_path = os.path.join(data_path, "data.zip")
    urllib.request.urlretrieve(data_url, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_path)

fetch_data()

def load_employee_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\employe", "employee_survey_data.csv")
    return pd.read_csv(csv_path)

def load_general_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\general", "general_data.csv")
    return pd.read_csv(csv_path)

def load_in_time_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\in_out_time", "in_time.csv")
    return pd.read_csv(csv_path)

def load_out_time_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\in_out_time", "out_time.csv")
    return pd.read_csv(csv_path)

def load_manager_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\manager", "manager_survey_data.csv")
    return pd.read_csv(csv_path)

employee = load_employee_data()
general = load_general_data()
in_time = load_in_time_data()
out_time = load_out_time_data()
manager = load_manager_data()

### 3. Analyse des différentes données

#### 3.1. employee

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Créez un pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
])


# Appliquez le pipeline à l'ensemble du DataFrame
employee_transformed = numeric_transformer.fit_transform(employee)

# Transformez le résultat du pipeline en DataFrame
employee_df = pd.DataFrame(employee_transformed, columns = employee.columns)
employee_df.info()

#### 3.2 General

| **Variable**           | **Type de données** | **Nombre de valeurs Manquantes** | **Nature de la variable** | **Nombre de catégories uniques** |
|------------------------|---------------------|----------------------------------|---------------------------|----------------------------------|
| **Age**                | Quantitative        | 0                                | Ordinal                   | 35                               |
| **Attrition**          | Qualitative         | 0                                | Nominal (Booléen)         | 2                                |
| **BusinessTravel**     | Qualitative         | 0                                | Ordinal                   | 3                                |
| **Department**         | Qualitative         | 0                                | Nominal                   | 3                                |
| **DistanceFromHome**   | Quantitative        | 0                                | Ordinal                   | 16                               |
| **Education**          | Quantitative        | 0                                | Ordinal                   | 5                                |
| **EducationField**     | Qualitative         | 0                                | Nominal                   | 6                                |
| **EmployeeCount**      | Quantitative        | 0                                | Ordinal                   | 1                                |
| **EmployeeID**         | Quantitative        | 0                                | Ordinal                   | 4410                             |
| **Gender**             | Qualitative         | 0                                | Nominal (Booléen)         | 2                                |
| **JobLevel**           | Quantitative        | 0                                | Ordinal                   | 5                                |
| **JobRole**            | Qualitative         | 0                                | Nominal                   | 9                                |
| **MaritalStatus**      | Qualitative         | 0                                | Nominal                   | 3                                |
| **MonthlyIncome**      | Quantitative        | 0                                | Ordinal                   | 1349                             |
| **NumCompaniesWorked** | Quantitative        | 19                               | Ordinal                   | 10                               |
| **Over18**             | Qualitative         | 0                                | Nominal                   | 1                                |
| **PercentSalaryHike**  | Quantitative        | 0                                | Ordinal                   | 14                               |
| **StandardHours**      | Quantitative        | 0                                | Ordinal                   | 1                                |
| **StockOptionLevel**   | Quantitative        | 0                                | Ordinal                   | 4                                |
| **TotalWorkingYears**  | Quantitative        | 9                                | Ordinal                   | 29                               |
| **TrainingTimesLastYear** | Quantitative      | 0                                | Ordinal                   | 7                                |
| **YearsAtCompany**     | Quantitative        | 0                                | Ordinal                   | 29                               |
| **YearsSinceLastPromotion** | Quantitative    | 0                                | Ordinal                   | 15                               |
| **YearsWithCurrManager** | Quantitative      | 0                                | Ordinal                   | 16                               |

In [ ]:
# Supprimer useless colonnes
general_dr = general.drop(['Over18', 'EmployeeCount', 'StandardHours'], axis=1)

# Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

general_num = general_dr.select_dtypes(include=[np.number]) 
num_attribs = list(general_num)

general_cat_ordinal = ['BusinessTravel']
general_cat_nominal = ['Attrition','Department','EducationField','Gender','JobRole','MaritalStatus']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
])

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat_nom", OneHotEncoder(), general_cat_nominal),
    ("cat_ord", OrdinalEncoder(), general_cat_ordinal),
])

# Appliquer le pipeline au DataFrame initial
general_tr = full_pipeline.fit_transform(general_dr)

# Créer un DataFrame à partir du résultat
general_prepared = pd.DataFrame(general_tr, columns=full_pipeline.get_feature_names_out())
general_prepared.drop(['cat_nom__Attrition_No'], axis=1)

#### 3.3 In_time

In [ ]:

def convert_all_to_datetime(df):
    year_dict = {}
    month_dict = {}
    day_dict = {}

    for column_name in df.columns:
        try:
            df[column_name] = pd.to_datetime(df[column_name])
            year_dict[column_name + '-year'] = df[column_name].dt.year
            month_dict[column_name + '-month'] = df[column_name].dt.month
            day_dict[column_name + '-day'] = df[column_name].dt.day
        except ValueError:
            # Skip columns that cannot be converted to datetime
            pass

    year_df = pd.DataFrame(year_dict)
    month_df = pd.DataFrame(month_dict)
    day_df = pd.DataFrame(day_dict)

    df = pd.concat([df, year_df, month_df, day_df], axis=1)
    return df

in_time = convert_all_to_datetime(in_time)
out_time = convert_all_to_datetime(out_time)

suprimmer les colones qui ne sont pas des dates

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def convert_all_to_datetime(X,  Y):    
    cols_to_drop = X.filter(regex='-day$').columns
    X = X.drop(cols_to_drop, axis=1)
    cols_to_drop = X.filter(regex='-month$').columns
    X = X.drop(cols_to_drop, axis=1)
    cols_to_drop = X.filter(regex='-year$').columns
    X = X.drop(cols_to_drop, axis=1)


    first_column = X.columns[0]
    X = X.drop([first_column], axis=1)
    # Supprimer les colonnes où toutes les valeurs sont NaT
    X = X.dropna(axis=1, how='all')

    cols_to_drop = Y.filter(regex='-day$').columns
    Y = Y.drop(cols_to_drop, axis=1)
    cols_to_drop = Y.filter(regex='-month$').columns
    Y = Y.drop(cols_to_drop, axis=1)
    cols_to_drop = Y.filter(regex='-year$').columns
    Y = Y.drop(cols_to_drop, axis=1)


    first_column = Y.columns[0]
    Y = Y.drop([first_column], axis=1)

    # Supprimer les colonnes où toutes les valeurs sont NaT
    Y = Y.dropna(axis=1, how='all')


    # Calculer la médiane du temps horaire pour chaque personne
    median_time = (Y - X).median()

    for column in X.columns:
        # Trouver où les valeurs sont manquantes
        in_time_nan = X[column].isna()
        out_time_nan = Y[column].isna()

        # Si 'in_time' est manquant, soustraire la médiane du temps horaire à 'out_time'
        X.loc[in_time_nan, column] = X.loc[in_time_nan, column] - median_time[column]

        # Si 'out_time' est manquant, ajouter la médiane du temps horaire à 'in_time'
        Y.loc[out_time_nan, column] = Y.loc[out_time_nan, column] + median_time[column]
    
    return  Y - X



class PreTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.imputer = SimpleImputer(strategy="median")

    def fit(self, X, y=None):
        self.imputer.fit(X)
        return self

    def transform(self, X, y=None):
        for col in X.columns:
            X[col] = X[col].apply(lambda x: x.total_seconds() / 3600 if pd.notnull(x) else np.nan)    
        stats = pd.DataFrame()
        stats['mean'] = pd.Series(X.mean(axis=1))
        stats['median'] = pd.Series(X.median(axis=1))
        stats['min'] = pd.Series(X.min(axis=1))
        stats['max'] = pd.Series(X.max(axis=1))
        return stats, stats.columns
        # Convertir les timedelta en une quantité numérique représentant le nombre d'heures


# Définir le pipeline
num_pipeline = Pipeline([
    ('pre_transformer', PreTransformer()),
])

# Appliquer le pipeline
hourly_time_prepared, stats_col = num_pipeline.fit_transform(convert_all_to_datetime(in_time, out_time))

hourly_time_prepared = pd.DataFrame(hourly_time_prepared, columns= stats_col )

#Ajouter la colonne 'EmployeeID' pour pouvoir joindre les DataFrames
hourly_time_prepared['EmployeeID'] = employee_df['EmployeeID']


#### Jointure

In [ ]:
# Utiliser la fonction merge pour faire la première jointure avec general
result = pd.merge(employee_df, manager, on='EmployeeID')

# Utiliser la fonction merge pour faire la deuxième jointure avec general_prepared
result = pd.merge(result, general_prepared, left_on='EmployeeID', right_on='num__EmployeeID')

# Utiliser la fonction merge pour faire la troisieme jointure avec hourly_time_prepared
result = pd.merge(result, hourly_time_prepared, on='EmployeeID')

result = result.drop(['EmployeeID'], axis=1)
result = result.drop(['num__EmployeeID'], axis=1)
# Créez un pipeline
result_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

result_array = result_transformer.fit_transform(result)
result = pd.DataFrame(result_array, columns = result.columns)


from sklearn.model_selection import train_test_split

# Supposons que vous voulez prédire 'cat_nom__Attrition_Yes'
y = result['cat_nom__Attrition_Yes']
X = result.drop(['cat_nom__Attrition_Yes', 'cat_nom__Attrition_No'], axis=1)

# Diviser les données en un ensemble d'entraînement et un ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_df = pd.DataFrame(pd.concat([X_train, y_train], axis = 1), columns=result.columns)


# Calculer la matrice de corrélation
corr_matrix = train_df.corr()
# Filtrer les corrélations
filtered_corr_matrix = corr_matrix.where((corr_matrix >= 0.4) | (corr_matrix <= -0.4))

# Trier la colonne 'cat_nom__Attrition_Yes' par valeur absolue
sorted_corr = corr_matrix['cat_nom__Attrition_Yes'].apply(abs).sort_values(ascending=False)

# Afficher les éléments les plus corrélés avec 'cat_nom__Attrition_Yes'
sorted_corr

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score


# Create a DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=42)

# Train the model
def train_model(trainer):
    trainer.fit(X_train, y_train)

    trainer_predictions = trainer.predict(X_test)
    trainer_mse = mean_squared_error(y_test, trainer_predictions)
    trainer_rmse = np.sqrt(trainer_mse)

    print("mse:", trainer_mse)
    print("rmse:", trainer_rmse)

    scores = cross_val_score(trainer, X_test, y_test,
                         scoring="neg_mean_squared_error", cv=10)
    trainer_rmse_scores = np.sqrt(-scores)
    
    print("Scores:", trainer_rmse_scores)
    print("Mean:", trainer_rmse_scores.mean())
    print("Standard deviation:", trainer_rmse_scores.std())


train_model(tree_reg)    








In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression(fit_intercept=False)
train_model(lin_reg)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(max_depth=15, max_features='log2', n_estimators=26,
                      random_state=42)
train_model(forest_reg)

In [ ]:
from sklearn.svm import SVR

svr_reg = SVR(C=9.318742350231167, gamma=0.09849250205191949)
train_model(svr_reg)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # essaye 12 (3×4) combinaisons des hyperparametres
    {'n_estimators': [3,7, 10,15,20,25 ,30], 'max_features': [2, 4, 6, 8]},
    # puis essaye 6 (2×3) combinaisons avec bootstrap à False (True étant la valeur par défaut)
    {'bootstrap': [False], 'n_estimators': [3,7, 10,15,20,25 ,30], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
 
# 5 sous-jeux de cross-val, ça fait en tout (12+6)*5=90 tours d'entraînement 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train,y_train )

print(grid_search.best_params_)
print(grid_search.best_estimator_)



In [ ]:
feature_importance = grid_search.best_estimator_.feature_importances_
attributes = result.columns
sorted(zip(feature_importance, attributes), reverse=True)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = [
    # essaye 12 (3×4) combinaisons des hyperparametres
    {'kernel': ['linear'], 'C': [10., 30., 100.]},
    # puis essaye 6 (2×3) combinaisons avec le noyau RBF
    {'kernel': ['rbf'], 'C': [10., 30., 100.],'gamma': [0.3, 1.0, 3.0]},
]

svr_reg = SVR()

# 5 sous-jeux de cross-val, ça fait en tout (8+6*6)*5=350 tours d'entraînement 
grid_search = GridSearchCV(svr_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', verbose=2, n_jobs=4)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

param_grid = [
    {'fit_intercept': [True, False]}
]

lin_reg = LinearRegression()

grid_search = GridSearchCV(lin_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

param_grid = [
    {'max_depth': [None, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 5]}
]

tree_reg = DecisionTreeRegressor(random_state=42)

grid_search = GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import uniform, randint

# Paramètres pour SVR
param_distributions_svr = {
    'C': uniform(0.1, 10),
    'gamma': uniform(0.01, 1),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

svr = SVR()

random_search_svr = RandomizedSearchCV(svr, param_distributions=param_distributions_svr, 
                                       n_iter=500, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search_svr.fit(X_train, y_train)

print(random_search_svr.best_params_)
print(random_search_svr.best_estimator_)

# Paramètres pour RandomForestRegressor
param_distributions_rf = {
    'n_estimators': randint(10, 200),
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': randint(2, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10)
}

rf = RandomForestRegressor(random_state=42)

random_search_rf = RandomizedSearchCV(rf, param_distributions=param_distributions_rf, 
                                      n_iter=500, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search_rf.fit(X_train, y_train)

print(random_search_rf.best_params_)
print(random_search_rf.best_estimator_)